In [1]:
import yfinance as yf
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

In [2]:
acao = yf.Ticker("RRRP3.SA")

In [29]:
acao = yf.download(tickers="RRRP3.SA", interval = "1m")


[*********************100%***********************]  1 of 1 completed


In [30]:
acao

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-11-24 10:03:00-03:00,35.900002,36.000000,35.900002,35.919998,35.919998,0
2022-11-24 10:04:00-03:00,35.930000,36.080002,35.930000,36.049999,36.049999,14900
2022-11-24 10:05:00-03:00,36.000000,36.020000,35.970001,35.970001,35.970001,4600
2022-11-24 10:06:00-03:00,36.029999,36.080002,36.029999,36.040001,36.040001,6300
2022-11-24 10:07:00-03:00,36.029999,36.029999,35.910000,36.029999,36.029999,27000
...,...,...,...,...,...,...
2022-11-30 15:56:00-03:00,35.869999,35.939999,35.849998,35.939999,35.939999,3700
2022-11-30 15:57:00-03:00,35.950001,35.950001,35.910000,35.950001,35.950001,3700
2022-11-30 15:58:00-03:00,35.959999,36.009998,35.959999,35.980000,35.980000,4800


In [13]:
teste = yf.download("PETR3.SA")

[*********************100%***********************]  1 of 1 completed


In [18]:
df = pd.DataFrame(acao['Close'])

#Médias móveis de 20 em 20 dias
mm = df.rolling(10).mean()

#Desvio padrão de 20 em 20 dias
std = df.rolling(10).std()

sup_band = mm + (2 * std)
inf_band = mm - (2.1 * std)

sup_band = sup_band.rename(columns={"Close":"Superior"})
inf_band = inf_band.rename(columns={"Close":"Inferior"})
mm = mm.rename(columns={"Close":"Medias"})
std = std.rename(columns={"Close":"Desvio padrao"})



bandas_bollinger = df.join(sup_band).join(inf_band)

bandas_bollinger.dropna(inplace=True)

compras = bandas_bollinger[bandas_bollinger["Close"]<=bandas_bollinger["Inferior"]]
vendas = bandas_bollinger[bandas_bollinger["Close"]>=bandas_bollinger["Superior"]]

compras = compras.rename(columns={"Close":"Compras"})
vendas = vendas.rename(columns={"Close":"Vendas"})

fig = go.Figure()

fig.add_trace(go.Scatter(
    x = inf_band.index,
    y = inf_band['Inferior'],
    name = "Banda inferior",
    line_color = "rgba(173,204,255,0.2)"
))

fig.add_trace(go.Scatter(
    x = sup_band.index,
    y = sup_band["Superior"],
    name = "Banda superior",
    fill="tonexty",
    line_color = "rgba(173,204,255,0.2)"
))

fig.add_trace(go.Scatter(
    x = df.index,
    y = df["Close"],
    name = "Preço de fechamento",
    line_color = "#636EFA"
))

fig.add_trace(go.Scatter(
    x = mm.index,
    y = mm["Medias"],
    name = "Média móvel",
    line_color = "#FECB52"
))

fig.add_trace(go.Scatter(
    x = compras.index,
    y = compras["Compras"],
    name = "Compra",
    mode = "markers",
    marker = dict(color="#00CC96",size=8)
))

fig.add_trace(go.Scatter(
    x = vendas.index,
    y = vendas["Vendas"],
    name = "Venda",
    mode = "markers",
    marker = dict(color="#EF553B",size = 8)
))


fig.show()

In [6]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    name='Fechamento', 
    x=df.index, 
    open=df["Open"], 
    high=df["High"],
    low=df["Low"],
    close=df["Close"]))